In [1]:
%matplotlib inline

In [2]:
from Bio import SeqIO, Phylo
import glob
import random

import pandas as pd
import numpy as np

# Just working with OMA groups first

In [ ]:
monophyly_df = pd.read_csv('../Data/OMA_group_data/eukaryotes/tree_monophyly.csv', index_col='Group_id')
print(monophyly_df.shape)
monophyly_df.head()

# Preserving the exact root and downstream clades

Initially, I ran this code to prune species in such a way as to **always** preserve the length of the root node. Which is to say, when pruning a single side/set I would ensure that the first bifurcation remained no matter what. Thus, I'd make sure to preserve at least two species on any given side when pruning so that this first split never "moved" and that the length of the target root sequence would always be the same no matter the pruning method.

In [ ]:
# target_mono_frac = 0.95
# for tree_loc in glob.glob('../Data/OMA_group_data/eukaryotes/'
#                           'processed_OMA_trees/*.RootedNoZero.MPAJH')[:]:
#     file_name = tree_loc.split('/')[-1]
#     print(file_name)
#     group = tree_loc.split('/')[-1].split('.')[0]
#     if type(monophyly_df.loc[group]['monophyletic_clade']) != str:
#         print('not mono')
#         continue
#     starting_tree = Phylo.read(tree_loc, 'newick')
#     starting_tree.root_with_outgroup(monophyly_df.loc[group]['other_clade'], outgroup_branch_length=0.0)
#     starting_tree.root_with_outgroup(monophyly_df.loc[group]['monophyletic_clade'], outgroup_branch_length=10e-10)
#     assert starting_tree.root.clades[1].name == monophyly_df.loc[group]['monophyletic_clade']
#     assert starting_tree.root.clades[0].name == monophyly_df.loc[group]['other_clade']
#     initial_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
#     ########
#     mono_ids = [term.name for term in starting_tree.root.clades[1].get_terminals()]
#     assert len(mono_ids) == len(starting_tree.root.clades[1].get_terminals())
#     other_ids = [term.name for term in starting_tree.root.clades[0].get_terminals()]
#     assert len(other_ids) == len(starting_tree.root.clades[0].get_terminals())
    
#     total_terms = len(mono_ids)+len(other_ids)
#     starting_mono_frac = len(mono_ids)/total_terms
#     if starting_mono_frac >= target_mono_frac:
#         clade_to_trim_from = 1
#         n_to_retain = target_mono_frac*total_terms
#         n_to_remove = round(len(mono_ids)-n_to_retain)
#     elif starting_mono_frac < target_mono_frac:
#         clade_to_trim_from = 0
#         n_to_retain = len(mono_ids)/target_mono_frac - len(mono_ids)
#         n_to_remove = round(len(other_ids)-n_to_retain)
#     if n_to_remove >= len(starting_tree.root.clades[clade_to_trim_from].get_terminals()):
#         print('With given parameters, you would remove all from clade. Leaving one member')
#         n_to_remove = len(starting_tree.root.clades[clade_to_trim_from].get_terminals())-1

#     ###I'm making this overly complicated here so that I don't ever accidentally remove the 
#     ###monophyletic node entirely! It iteratively removes one at a time and does not allow for
#     ###removal of a single clade connected to the monophyletic node
#     for i in range(n_to_remove):
#         #######
#         l_terms = starting_tree.root.clades[clade_to_trim_from].clades[0].get_terminals()
#         r_terms = starting_tree.root.clades[clade_to_trim_from].clades[1].get_terminals()
#         #######
#         if len(l_terms) > 1 and len(r_terms) > 1:
#             combined_terms = l_terms + r_terms
#         elif len(l_terms) > 1 and len(r_terms) == 1:
#             combined_terms = l_terms
#         elif len(l_terms) == 1 and len(r_terms) > 1:
#             combined_terms = r_terms
#         else:
#             print('Something unexpected is happening during node removal and you should investigate')
#             print(len(l_terms), len(r_terms))
#             break
#         term_to_remove = random.choice(combined_terms)
#         starting_tree.prune(term_to_remove)
#     final_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
#     assert initial_root_dist == final_root_dist
#     outfile_loc = tree_loc.replace('processed_OMA_trees/', 'pruned_OMA_trees/')
#     outfile_loc = outfile_loc.replace(file_name, 
#                                       '{}.frac{}.treefile'.format(group, target_mono_frac))
#     Phylo.write(starting_tree, outfile_loc, 'newick', format_branch_length='%1.16f')

# Removing the restriction

Now, this code will remove any/all species from a given side without regard to who/the first bifurcation

In [ ]:
target_mono_frac = 0.5
for tree_loc in glob.glob('../Data/OMA_group_data/eukaryotes/'
                          'processed_OMA_trees/*.RootedNoZero.MPAJH')[:]:
    file_name = tree_loc.split('/')[-1]
    print(file_name)
    group = tree_loc.split('/')[-1].split('.')[0]
    if type(monophyly_df.loc[group]['monophyletic_clade']) != str:
        print('not mono')
        continue
    starting_tree = Phylo.read(tree_loc, 'newick')
    starting_tree.root_with_outgroup(monophyly_df.loc[group]['other_clade'], outgroup_branch_length=0.0)
    starting_tree.root_with_outgroup(monophyly_df.loc[group]['monophyletic_clade'], outgroup_branch_length=10e-10)
    assert starting_tree.root.clades[1].name == monophyly_df.loc[group]['monophyletic_clade']
    assert starting_tree.root.clades[0].name == monophyly_df.loc[group]['other_clade']
    initial_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
    ########
    mono_ids = [term.name for term in starting_tree.root.clades[1].get_terminals()]
    assert len(mono_ids) == len(starting_tree.root.clades[1].get_terminals())
    other_ids = [term.name for term in starting_tree.root.clades[0].get_terminals()]
    assert len(other_ids) == len(starting_tree.root.clades[0].get_terminals())
    
    total_terms = len(mono_ids)+len(other_ids)
    starting_mono_frac = len(mono_ids)/total_terms
    if starting_mono_frac >= target_mono_frac:
        clade_to_trim_from = 1
        n_to_retain = round((target_mono_frac*len(other_ids))/(1-target_mono_frac))
        n_to_remove = round(len(mono_ids)-n_to_retain)
    elif starting_mono_frac < target_mono_frac:
        clade_to_trim_from = 0
        n_to_retain = round((len(mono_ids) - (target_mono_frac*len(mono_ids)))/target_mono_frac)
        n_to_remove = round(len(other_ids)-n_to_retain)
    if n_to_remove >= len(starting_tree.root.clades[clade_to_trim_from].get_terminals()):
        print('With given parameters, you would remove all from clade. Leaving one member')
        n_to_remove = len(starting_tree.root.clades[clade_to_trim_from].get_terminals())-1

    for i in range(n_to_remove):
        my_terms = starting_tree.root.clades[clade_to_trim_from].get_terminals()
        term_to_remove = random.choice(my_terms)
        starting_tree.prune(term_to_remove)
    final_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
    outfile_loc = tree_loc.replace('processed_OMA_trees/', 'pruned_OMA_trees/')
    outfile_loc = outfile_loc.replace(file_name, 
                                      '{}.frac{}.treefile'.format(group, target_mono_frac))
    print(len(starting_tree.root.clades[0].get_terminals())/len(starting_tree.get_terminals()))
    print(len(starting_tree.root.clades[1].get_terminals())/len(starting_tree.get_terminals()))
    Phylo.write(starting_tree, outfile_loc, 'newick', format_branch_length='%1.16f')

# And a similar thing for the Tria *et al.* dataset 

In [4]:
monophyly_df = pd.read_csv('../Data/Tria_et_al_data/eukaryotes/tree_monophyly.csv', index_col='Group_id')
print(monophyly_df.shape)
monophyly_df.head()

(1446, 9)


,monophyletic_clade,other_clade,root_bl,monophyletic_total_bl,other_total_bl,total_tree_bl,total_n,monophyletic_n,other_n
Group_id,,,,,,,,,
KOG2750,IntNode_5,IntNode_9,1.25880,19.950620,23.824960,45.034380,31.0,17.0,14.0
KOG3183,IntNode_13,IntNode_3,0.68573,8.948311,9.585423,19.219464,31.0,17.0,14.0
KOG2581,IntNode_3,IntNode_16,0.41771,7.610274,5.181570,13.209554,31.0,17.0,14.0
KOG0728,IntNode_1,IntNode_2,0.36531,1.707108,1.102690,3.175108,31.0,17.0,14.0
KOG0157,IntNode_13,IntNode_6,2.17420,3.997834,12.613323,18.785357,31.0,17.0,14.0


# The old way...

In [ ]:
# target_mono_frac = 0.9
# for tree_loc in glob.glob('../Data/Tria_et_al_data/eukaryotes/'
#                           'processed_trees/*.RootedNoZero.MPAJH')[:]:
#     file_name = tree_loc.split('/')[-1]
#     print(file_name)
#     group = tree_loc.split('/')[-1].split('.')[0]
#     if type(monophyly_df.loc[group]['monophyletic_clade']) != str:
#         print('not mono')
#         continue
#     starting_tree = Phylo.read(tree_loc, 'newick')
#     starting_tree.root_with_outgroup(monophyly_df.loc[group]['other_clade'], outgroup_branch_length=0.0)
#     starting_tree.root_with_outgroup(monophyly_df.loc[group]['monophyletic_clade'], outgroup_branch_length=10e-10)
#     assert starting_tree.root.clades[1].name == monophyly_df.loc[group]['monophyletic_clade']
#     assert starting_tree.root.clades[0].name == monophyly_df.loc[group]['other_clade']
#     initial_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
#     ########
#     mono_ids = [term.name for term in starting_tree.root.clades[1].get_terminals()]
#     assert len(mono_ids) == len(starting_tree.root.clades[1].get_terminals())
#     other_ids = [term.name for term in starting_tree.root.clades[0].get_terminals()]
#     assert len(other_ids) == len(starting_tree.root.clades[0].get_terminals())
    
#     total_terms = len(mono_ids)+len(other_ids)
#     starting_mono_frac = len(mono_ids)/total_terms
#     if starting_mono_frac >= target_mono_frac:
#         clade_to_trim_from = 1
#         n_to_retain = target_mono_frac*total_terms
#         n_to_remove = round(len(mono_ids)-n_to_retain)
#     elif starting_mono_frac < target_mono_frac:
#         clade_to_trim_from = 0
#         n_to_retain = len(mono_ids)/target_mono_frac - len(mono_ids)
#         n_to_remove = round(len(other_ids)-n_to_retain)
#     if n_to_remove >= len(starting_tree.root.clades[clade_to_trim_from].get_terminals()):
#         print('With given parameters, you would remove all from clade. Leaving one member')
#         n_to_remove = len(starting_tree.root.clades[clade_to_trim_from].get_terminals())-1

#     ###I'm making this overly complicated here so that I don't ever accidentally remove the 
#     ###monophyletic node entirely! It iteratively removes one at a time and does not allow for
#     ###removal of a single clade connected to the monophyletic node
#     for i in range(n_to_remove):
#         #######
#         l_terms = starting_tree.root.clades[clade_to_trim_from].clades[0].get_terminals()
#         r_terms = starting_tree.root.clades[clade_to_trim_from].clades[1].get_terminals()
#         #######
#         if len(l_terms) > 1 and len(r_terms) > 1:
#             combined_terms = l_terms + r_terms
#         elif len(l_terms) > 1 and len(r_terms) == 1:
#             combined_terms = l_terms
#         elif len(l_terms) == 1 and len(r_terms) > 1:
#             combined_terms = r_terms
#         else:
#             print('Something unexpected is happening during node removal and you should investigate')
#             print(len(l_terms), len(r_terms))
#             break
#         term_to_remove = random.choice(combined_terms)
#         starting_tree.prune(term_to_remove)
#     final_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
#     assert initial_root_dist == final_root_dist
#     outfile_loc = tree_loc.replace('processed_trees/', 'pruned_trees/')
#     outfile_loc = outfile_loc.replace(file_name, 
#                                       '{}.frac{}.treefile'.format(group, target_mono_frac))
#     print(len(starting_tree.root.clades[1].get_terminals())/len(starting_tree.get_terminals()))
#     Phylo.write(starting_tree, outfile_loc, 'newick', format_branch_length='%1.16f')

# And the new way

In [17]:
target_mono_frac = 0.05
for tree_loc in glob.glob('../Data/Tria_et_al_data/eukaryotes/'
                          'processed_trees/*.RootedNoZero.MPAJH')[:]:
    file_name = tree_loc.split('/')[-1]
    print(file_name)
    group = tree_loc.split('/')[-1].split('.')[0]
    if type(monophyly_df.loc[group]['monophyletic_clade']) != str:
        print('not mono')
        continue
    starting_tree = Phylo.read(tree_loc, 'newick', rooted=True)
    starting_tree.root_with_outgroup(monophyly_df.loc[group]['other_clade'], outgroup_branch_length=0.0)
    starting_tree.root_with_outgroup(monophyly_df.loc[group]['monophyletic_clade'], outgroup_branch_length=10e-10)
    assert starting_tree.root.clades[1].name == monophyly_df.loc[group]['monophyletic_clade']
    assert starting_tree.root.clades[0].name == monophyly_df.loc[group]['other_clade']
    initial_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
    ########
    mono_ids = [term.name for term in starting_tree.root.clades[1].get_terminals()]
    assert len(mono_ids) == len(starting_tree.root.clades[1].get_terminals())
    other_ids = [term.name for term in starting_tree.root.clades[0].get_terminals()]
    assert len(other_ids) == len(starting_tree.root.clades[0].get_terminals())
    
    total_terms = len(mono_ids)+len(other_ids)
    starting_mono_frac = len(mono_ids)/total_terms
    if starting_mono_frac >= target_mono_frac:
        clade_to_trim_from = 1
        n_to_retain = round((target_mono_frac*len(other_ids))/(1-target_mono_frac))
        n_to_remove = len(mono_ids)-n_to_retain


    elif starting_mono_frac < target_mono_frac:
        clade_to_trim_from = 0
        n_to_retain = round((len(mono_ids) - (target_mono_frac*len(mono_ids)))/target_mono_frac)
        n_to_remove = round(len(other_ids)-n_to_retain)
        
    if n_to_remove >= len(starting_tree.root.clades[clade_to_trim_from].get_terminals()):
        print('With given parameters, you would remove all from clade. Leaving one member')
        n_to_remove = len(starting_tree.root.clades[clade_to_trim_from].get_terminals())-1

    for i in range(n_to_remove):
        my_terms = starting_tree.root.clades[clade_to_trim_from].get_terminals()
        term_to_remove = random.choice(my_terms)
        starting_tree.prune(term_to_remove)
        
    final_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
    outfile_loc = tree_loc.replace('processed_trees/', 'pruned_trees/')
    outfile_loc = outfile_loc.replace(file_name, 
                                      '{}.frac{}.treefile'.format(group, target_mono_frac))
    print(len(starting_tree.root.clades[0].get_terminals())/len(starting_tree.get_terminals()))
    print(len(starting_tree.root.clades[1].get_terminals())/len(starting_tree.get_terminals()))
    Phylo.write(starting_tree, outfile_loc, 'newick', format_branch_length='%1.16f')

KOG2750.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3183.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2581.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0728.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0157.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2950.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1652.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1122.treefile.RootedNoZero.MPAJH
not mono
KOG1511.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2826.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3139.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3628.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3185.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3893.treefile.RootedNoZero.MPAJH
0.9333333333333333


KOG2078.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1825.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0526.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2948.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0334.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1444.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0687.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3037.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1376.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0068.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1211.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0864.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2330.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1462.treefile.RootedNo

KOG3911.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2473.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2646.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2006.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2044.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1607.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0534.treefile.RootedNoZero.MPAJH
not mono
KOG2140.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2189.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0057.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0285.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3071.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2168.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0667.treefile.RootedNoZero.MPAJH
0.9333333333333333


0.9333333333333333
0.06666666666666667
KOG1253.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2100.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3393.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2866.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0058.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0504.treefile.RootedNoZero.MPAJH
not mono
KOG3480.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2309.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3499.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3087.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0894.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0048.treefile.RootedNoZero.MPAJH
not mono
KOG2115.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1415.treefile.RootedNoZero.MPAJH
0.93333333

KOG0190.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2583.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1692.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0892.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3051.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1656.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3907.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2780.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2249.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1329.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1919.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0729.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0856.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1973.treefile.RootedNo

...

KOG2196.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3880.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1290.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0330.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2917.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2737.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2058.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1921.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0015.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1145.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1311.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2570.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0761.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1615.treefile.RootedNo

KOG3079.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1943.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3996.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2540.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2054.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0274.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0779.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1430.treefile.RootedNoZero.MPAJH
not mono
KOG0748.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0659.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1241.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3123.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2051.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1961.treefile.RootedNoZero.MPAJH
0.9333333333333333


KOG2478.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0416.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3052.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2042.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG4031.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1558.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0363.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1332.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0251.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3689.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2411.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1835.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1720.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0490.treefile.RootedNo

KOG0208.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2513.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0188.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1014.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2167.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1491.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2273.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1671.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2403.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0103.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3741.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3303.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2210.treefile.RootedNoZero.MPAJH
not mono
KOG3199.treefile.RootedNoZero.MPAJH
0.9333333333333333


KOG1900.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1506.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2845.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1860.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0227.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2504.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG0537.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG4075.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG2757.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3038.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3712.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1390.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG3754.treefile.RootedNoZero.MPAJH
0.9333333333333333
0.06666666666666667
KOG1062.treefile.RootedNo